# Experiment 1: the discrimination of genuine and artificially shuffled full attentional vectors (Fig 2)

In [ ]:
# To run the notebook, change the string below
dir_path = '/path/to/my/directory'

# Uncomment to run in Google Colab
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
import sys
sys.path.append(dir_path)

# ViT model based on https://github.com/facebookresearch/dino/blob/main/README.md
import importlib
import torch
import torch.random
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.utils.data as data_utils
import numpy as np
from PIL import Image
import vision_transformer
import fnmatch
import os
import glob
import shutil
import matplotlib.pyplot as plt
from copy import deepcopy
device = torch.device("cuda")

# IMAGE CLASSIFICATION A, B, AND C DATA LOADERS

traindirA = dir_path+"/data/train/classificationA"
valdirA = dir_path+"/data/val/classificationA"

traindirB = dir_path+"/data/train/classificationB"
valdirB = dir_path+"/data/val/classificationB"

traindirC = dir_path+"/data/train/classificationC"
valdirC = dir_path+"/data/val/classificationC"

train_transforms = transforms.Compose([transforms.Resize((256,256)),
                                       transforms.ToTensor(),
                                       ])
val_transforms = transforms.Compose([transforms.Resize((256,256)),
                                      transforms.ToTensor(),
                                      ])


def schematrain(model, x, y, optimizer):
    pred_attn, h1m, policy = model.forward(x)
    mse = torch.nn.MSELoss()
    bce = torch.nn.BCEWithLogitsLoss()
    pred_loss = 0.05*mse(pred_attn, h1m)
    policy_loss = bce(policy, y)
    total_loss = sum([pred_loss, policy_loss])
    total_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return total_loss

def schematrain_policy(model, x, y, optimizer):
    pred_attn, h1m, policy = model.forward(x)
    bce = torch.nn.BCEWithLogitsLoss()
    policy_loss = bce(policy, y)
    policy_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return policy_loss

def controltrain(model, x, y, optimizer):
    h1, policy = model.forward(x)
    bce = torch.nn.BCEWithLogitsLoss()
    policy_loss = bce(policy, y)
    policy_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return policy_loss

def fitschema(model, trainloader, valloader, name="", n_epochs=20, policy_only=False):
  bce = torch.nn.BCEWithLogitsLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
  losses = []
  val_losses = []

  epoch_train_losses = []
  epoch_val_losses = []

  for epoch in range(n_epochs):
      epoch_loss = 0
      for i, data in enumerate(trainloader): # iterate over batches
          x_batch, y_batch = data
          x_batch, y_batch = x_batch.to(device), y_batch.to(device)
          y_batch = y_batch.unsqueeze(1).float()
          model.train()
          if policy_only:
            loss = schematrain_policy(model, x_batch, y_batch, optimizer)
          else:
            loss = schematrain(model, x_batch, y_batch, optimizer)
          epoch_loss += loss.item()/len(trainloader)
          losses.append(loss.item())
          if epoch == 0:
            print(str(i)+": "+str(loss.item())+" / "+str(len(trainloader))+": "+str(epoch_loss))
      epoch_train_losses.append(epoch_loss)
      print('\nEpoch : {}, train loss : {}'.format(epoch+1,epoch_loss))
      with torch.no_grad():
        total_loss = 0
        for x_batch, y_batch in valloader:
          x_batch = x_batch.to(device)
          y_batch = y_batch.unsqueeze(1).float()
          y_batch = y_batch.to(device)

          model.eval()

          _, _, policy = model(x_batch)
          val_loss = bce(policy,y_batch)
          total_loss += val_loss.item()/len(valloader)
          val_losses.append(val_loss.item())

        epoch_val_losses.append(total_loss)
        print('Epoch : {}, val loss : {}'.format(epoch+1,total_loss))

        best_loss = min(epoch_val_losses)

        # save best model
        if total_loss <= best_loss:
          best_model_wts = model.state_dict()

  model.load_state_dict(best_model_wts)

def fitcontrol(model, trainloader, valloader, name="", n_epochs=20):
  bce = torch.nn.BCEWithLogitsLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

  losses = []
  val_losses = []

  epoch_train_losses = []
  epoch_val_losses = []

  for epoch in range(n_epochs):
      epoch_loss = 0
      for i, data in enumerate(trainloader): #iterate over batches
          x_batch, y_batch = data
          x_batch, y_batch = x_batch.to(device), y_batch.to(device)
          y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
          model.train()
          loss = controltrain(model, x_batch, y_batch, optimizer)
          epoch_loss += loss.item()/len(trainloader)
          losses.append(loss.item())
          if epoch == 0:
            print(str(i)+": "+str(loss.item())+" / "+str(len(trainloader))+": "+str(epoch_loss))
      epoch_train_losses.append(epoch_loss)
      print('\nEpoch : {}, train loss : {}'.format(epoch+1,epoch_loss))
      with torch.no_grad():
        total_loss = 0
        for x_batch, y_batch in valloader:
          x_batch = x_batch.to(device)
          y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
          y_batch = y_batch.to(device)

          #model to eval mode
          model.eval()

          _, policy = model(x_batch)
          val_loss = bce(policy,y_batch)
          total_loss += val_loss.item()/len(valloader)
          val_losses.append(val_loss.item())

        epoch_val_losses.append(total_loss)
        print('Epoch : {}, val loss : {}'.format(epoch+1,total_loss))

        best_loss = min(epoch_val_losses)

        #save best model
        if total_loss <= best_loss:
          best_model_wts = model.state_dict()

  model.load_state_dict(best_model_wts)

def evaluate(model, valloader, name="", save_attn=False):
  classifications = []
  labels = []
  model.eval()
  sigmoid = torch.nn.Sigmoid()
  total_acc = 0
  for i, data in enumerate(valloader):
      accuracy = 0
      x_batch, y_batch = data
      x_batch, y_batch = x_batch.to(device), y_batch.to(device)
      y_batch = y_batch.unsqueeze(1).float()
      outputs = model.forward(x_batch)
      policy = outputs[-1]
      policy = torch.round(sigmoid(policy))
      accuracy = 1-(torch.sum(abs(policy - y_batch))/len(y_batch))
      total_acc += accuracy.item()/len(valloader)
      for pol in policy:
        classifications.append(pol.item())
      for yb in y_batch:
        labels.append(yb.item())

  file = open(dir_path+"/accuracy_"+name+".txt","w")
  file.write(str(total_acc))
  file.close()

def freeze_models(models):
  for i, model in enumerate(models):
    for param in model.parameters():
        param.requires_grad = False
    for param in model.policy.parameters():
        param.requires_grad = True


In [ ]:
seedn = 0
seeds = [72442,16007,15137,96512,19047,59485,75241,95430,72796,63453,26884,53675,
         18008,15186,27656,31995,93321,89984,29108,75579,35223,13737,92478,17877,
         68783,67243,71062,45080,43868,38000,73096,51761,64413,62026,50615,23993,
         50152,22721,92064,87461,97294,59936,14695,15888,48874,37701,27120,60244,
         97999,73735,81996,72191,77250,50393,23720,63282,19530,45563,98929,14856,
         78783,75455,55985,89396,74140,74802,58912,14247,13741,41605,94482,20021,
         94900,54095,56975,57805,76423,58744,22887,62985,29424,58566,19647,65836,
         49274,99511,81839,78935,29560,97097,85628,87836,69055,19863,38173,80205,
         25417,79727,92203,69116]

for trial in range(11):
  torch.manual_seed(seeds[seedn])
  tname = "trial"+str(trial)

  # FIT / EVALUATE MODELS: IMAGE CLASSIFICATION
  # A
  train_dataA = datasets.ImageFolder(traindirA,transform=train_transforms)
  val_dataA = datasets.ImageFolder(valdirA,transform=val_transforms)

  trainloaderA = torch.utils.data.DataLoader(train_dataA, shuffle = True, batch_size=8)
  valloaderA = torch.utils.data.DataLoader(val_dataA, shuffle = True, batch_size=8)

  modelAschema = vision_transformer.VitAttentionSchema().to(device)
  modelAcontrol = vision_transformer.VitControl().to(device)

  fitschema(modelAschema, trainloaderA, valloaderA, tname+"A")
  fitcontrol(modelAcontrol, trainloaderA, valloaderA, tname+"A")

  evaluate(modelAschema, valloaderA, tname+"Aschema", save_attn=False)
  evaluate(modelAcontrol, valloaderA, tname+"Acontrol", save_attn=False)

  del(train_dataA)
  del(val_dataA)
  del(trainloaderA)
  del(valloaderA)

  # B
  train_dataB = datasets.ImageFolder(traindirB,transform=train_transforms)
  val_dataB = datasets.ImageFolder(valdirB,transform=val_transforms)

  trainloaderB = torch.utils.data.DataLoader(train_dataB, shuffle = True, batch_size=8)
  valloaderB = torch.utils.data.DataLoader(val_dataB, shuffle = True, batch_size=8)

  modelBschema = vision_transformer.VitAttentionSchema().to(device)
  modelBcontrol = vision_transformer.VitControl().to(device)

  fitschema(modelBschema, trainloaderB, valloaderB, tname+"B")
  fitcontrol(modelBcontrol, trainloaderB, valloaderB, tname+"B")

  evaluate(modelBschema, valloaderB, tname+"Bschema", save_attn=False)
  evaluate(modelBcontrol, valloaderB, tname+"Bcontrol", save_attn=False)

  del(train_dataB)
  del(val_dataB)
  del(trainloaderB)
  del(valloaderB)

  # C
  train_dataC = datasets.ImageFolder(traindirC,transform=train_transforms)
  val_dataC = datasets.ImageFolder(valdirC,transform=val_transforms)

  trainloaderC = torch.utils.data.DataLoader(train_dataC, shuffle = True, batch_size=8)
  valloaderC = torch.utils.data.DataLoader(val_dataC, shuffle = True, batch_size=8)

  modelCschema = vision_transformer.VitAttentionSchema().to(device)
  modelCcontrol = vision_transformer.VitControl().to(device)
  fitschema(modelCschema, trainloaderC, valloaderC, tname+"C")
  fitcontrol(modelCcontrol, trainloaderC, valloaderC, tname+"C")

  evaluate(modelCschema, valloaderC, tname+"Cschema", save_attn=False)
  evaluate(modelCcontrol, valloaderC, tname+"Ccontrol", save_attn=False)

  del(train_dataC)
  del(val_dataC)
  del(trainloaderC)
  del(valloaderC)

  # FREEZE THE MODELS
  freeze_models([modelAschema, modelAcontrol, modelBschema, modelBcontrol, modelCschema, modelCcontrol])

  modelAschema_wts = deepcopy(modelAschema.state_dict())
  modelAcontrol_wts = deepcopy(modelAcontrol.state_dict())
  modelBschema_wts = deepcopy(modelBschema.state_dict())
  modelBcontrol_wts = deepcopy(modelBcontrol.state_dict())
  modelCschema_wts = deepcopy(modelCschema.state_dict())
  modelCcontrol_wts = deepcopy(modelCcontrol.state_dict())

  # FIT / EVALUATE MODELS: ATTENTION CLASSIFICATION
  # SCHEMA
  batch_size = 8
  attn_transforms = transforms.Resize((256,256))

  schemaAattn = torch.load(dir_path+"/data/attentions/modelAschemaattn.pt")
  schemaAattn = attn_transforms(schemaAattn)
  false_schemaAattn = torch.clone(schemaAattn)
  indices = torch.randperm(false_schemaAattn.shape[-1])
  false_schemaAattn = false_schemaAattn[:,:,indices] # false attention values are shuffled along last dimension

  dataset_schemaAattn = TensorDataset(torch.cat((schemaAattn, false_schemaAattn),0),
                                      torch.cat((torch.ones(730,), torch.zeros(730,)),0))
  dataset_schemaAattn = torch.utils.data.random_split(dataset_schemaAattn, [0.9, 0.1])
  schemaAattntrain = DataLoader(dataset_schemaAattn[0], batch_size, shuffle=True)
  schemaAattnval = DataLoader(dataset_schemaAattn[1], batch_size, shuffle=True)

  fitcontrol(modelCcontrol, schemaAattntrain, schemaAattnval, tname+"CAcontrol_schemaattn", n_epochs=800)
  evaluate(modelCcontrol, schemaAattnval, tname+"CAcontrol_schemaattn", save_attn=False)
  fitschema(modelCschema, schemaAattntrain, schemaAattnval, tname+"CAschema_schemaattn", n_epochs=800, policy_only=True)
  evaluate(modelCschema, schemaAattnval, tname+"CAschema_schemaattn", save_attn=False)

  seedn += 1
  torch.manual_seed(seeds[seedn])

  del(schemaAattn)
  del(false_schemaAattn)
  del(indices)
  del(dataset_schemaAattn)
  del(schemaAattntrain)
  del(schemaAattnval)

  schemaBattn = torch.load(dir_path+"/data/attentions/modelBschemaattn.pt")
  schemaBattn = attn_transforms(schemaBattn)
  false_schemaBattn = torch.clone(schemaBattn)
  indices = torch.randperm(false_schemaBattn.shape[-1])
  false_schemaBattn = false_schemaBattn[:,:,indices]

  dataset_schemaBattn = TensorDataset(torch.cat((schemaBattn, false_schemaBattn),0),
                                      torch.cat((torch.ones(730,), torch.zeros(730,)),0))
  dataset_schemaBattn = torch.utils.data.random_split(dataset_schemaBattn, [0.9, 0.1])
  schemaBattntrain = DataLoader(dataset_schemaBattn[0], batch_size, shuffle=True)
  schemaBattnval = DataLoader(dataset_schemaBattn[1], batch_size, shuffle=True)

  fitschema(modelAschema, schemaBattntrain, schemaBattnval, tname+"ABschema_schemaattn", n_epochs=800, policy_only=True)
  fitcontrol(modelAcontrol, schemaBattntrain, schemaBattnval, tname+"ABcontrol_schemaattn", n_epochs=800)

  evaluate(modelAschema, schemaBattnval, tname+"ABschema_schemaattn", save_attn=False)
  evaluate(modelAcontrol, schemaBattnval, tname+"ABcontrol_schemaattn", save_attn=False)

  seedn += 1
  torch.manual_seed(seeds[seedn])

  del(schemaBattn)
  del(false_schemaBattn)
  del(indices)
  del(dataset_schemaBattn)
  del(schemaBattntrain)
  del(schemaBattnval)

  schemaCattn = torch.load(dir_path+"/data/attentions/modelCschemaattn.pt")
  schemaCattn = attn_transforms(schemaCattn)
  false_schemaCattn = torch.clone(schemaCattn)
  indices = torch.randperm(false_schemaCattn.shape[-1])
  false_schemaCattn = false_schemaCattn[:,:,indices]

  dataset_schemaCattn = TensorDataset(torch.cat((schemaCattn, false_schemaCattn),0),
                                      torch.cat((torch.ones(730,), torch.zeros(730,)),0))
  dataset_schemaCattn = torch.utils.data.random_split(dataset_schemaCattn, [0.9, 0.1])
  schemaCattntrain = DataLoader(dataset_schemaCattn[0], batch_size, shuffle=True)
  schemaCattnval = DataLoader(dataset_schemaCattn[1], batch_size, shuffle=True)

  fitschema(modelBschema, schemaCattntrain, schemaCattnval, "BCschema_schemaattn", n_epochs=800, policy_only=True)
  fitcontrol(modelBcontrol, schemaCattntrain, schemaCattnval, "BCcontrol_schemaattn", n_epochs=800)

  evaluate(modelBschema, schemaCattnval, tname+"BCschema_schemaattn", save_attn=False)
  evaluate(modelBcontrol, schemaCattnval, tname+"BCcontrol_schemaattn", save_attn=False)

  seedn += 1
  torch.manual_seed(seeds[seedn])

  del(schemaCattn)
  del(false_schemaCattn)
  del(indices)
  del(dataset_schemaCattn)
  del(schemaCattntrain)
  del(schemaCattnval)

  # CONTROL
  del(modelAschema)
  del(modelAcontrol)
  del(modelBschema)
  del(modelBcontrol)
  del(modelCschema)
  del(modelCcontrol)

  modelCschema = vision_transformer.VitAttentionSchema().to(device)
  modelCcontrol = vision_transformer.VitControl().to(device)
  modelCschema.load_state_dict(modelCschema_wts)
  modelCcontrol.load_state_dict(modelCcontrol_wts)
  freeze_models([modelCschema, modelCcontrol])

  controlAattn = torch.load(dir_path+"/data/attentions/modelAcontrolattn.pt")
  controlAattn = attn_transforms(controlAattn)
  false_controlAattn = torch.clone(controlAattn)
  indices = torch.randperm(false_controlAattn.shape[-1])
  false_controlAattn = false_controlAattn[:,:,indices] # false attention values are shuffled along last dimension

  dataset_controlAattn = TensorDataset(torch.cat((controlAattn, false_controlAattn),0),
                                      torch.cat((torch.ones(730,), torch.zeros(730,)),0))
  dataset_controlAattn = torch.utils.data.random_split(dataset_controlAattn, [0.9, 0.1])
  controlAattntrain = DataLoader(dataset_controlAattn[0], batch_size, shuffle=True)
  controlAattnval = DataLoader(dataset_controlAattn[1], batch_size, shuffle=True)

  fitschema(modelCschema, controlAattntrain, controlAattnval, tname+"CAs_controlattn", n_epochs=800, policy_only=True)
  fitcontrol(modelCcontrol, controlAattntrain, controlAattnval, tname+"CAc_controlattn", n_epochs=800)

  evaluate(modelCschema, controlAattnval, tname+"CAschema_controlattn", save_attn=False)
  evaluate(modelCcontrol, controlAattnval, tname+"CAcontrol_controlattn", save_attn=False)

  seedn += 1
  torch.manual_seed(seeds[seedn])

  del(controlAattn)
  del(false_controlAattn)
  del(indices)
  del(dataset_controlAattn)
  del(controlAattntrain)
  del(controlAattnval)
  del(modelCschema)
  del(modelCcontrol)
  del(modelCschema_wts)
  del(modelCcontrol_wts)

  modelAschema = vision_transformer.VitAttentionSchema().to(device)
  modelAcontrol = vision_transformer.VitControl().to(device)
  modelAschema.load_state_dict(modelAschema_wts)
  modelAcontrol.load_state_dict(modelAcontrol_wts)
  freeze_models([modelAschema, modelAcontrol])

  controlBattn = torch.load(dir_path+"/data/attentions/modelBcontrolattn.pt")
  controlBattn = attn_transforms(controlBattn)
  false_controlBattn = torch.clone(controlBattn)
  indices = torch.randperm(false_controlBattn.shape[-1])
  false_controlBattn = false_controlBattn[:,:,indices]

  dataset_controlBattn = TensorDataset(torch.cat((controlBattn, false_controlBattn),0),
                                      torch.cat((torch.ones(730,), torch.zeros(730,)),0))
  dataset_controlBattn = torch.utils.data.random_split(dataset_controlBattn, [0.9, 0.1])
  controlBattntrain = DataLoader(dataset_controlBattn[0], batch_size, shuffle=True)
  controlBattnval = DataLoader(dataset_controlBattn[1], batch_size, shuffle=True)

  fitschema(modelAschema, controlBattntrain, controlBattnval, tname+"ABs_controlattn", n_epochs=800, policy_only=True)
  fitcontrol(modelAcontrol, controlBattntrain, controlBattnval, tname+"ABc_controlattn", n_epochs=800)

  evaluate(modelAschema, controlBattnval, tname+"ABschema_controlattn", save_attn=False)
  evaluate(modelAcontrol, controlBattnval, tname+"ABcontrol_controlattn", save_attn=False)

  seedn += 1
  torch.manual_seed(seeds[seedn])

  del(controlBattn)
  del(false_controlBattn)
  del(indices)
  del(dataset_controlBattn)
  del(controlBattntrain)
  del(controlBattnval)
  del(modelAschema)
  del(modelAcontrol)
  del(modelAschema_wts)
  del(modelAcontrol_wts)

  modelBschema = vision_transformer.VitAttentionSchema().to(device)
  modelBcontrol = vision_transformer.VitControl().to(device)
  modelBschema.load_state_dict(modelBschema_wts)
  modelBcontrol.load_state_dict(modelBcontrol_wts)
  freeze_models([modelBschema, modelBcontrol])

  controlCattn = torch.load(dir_path+"/data/attentions/modelCcontrolattn.pt")
  controlCattn = attn_transforms(controlCattn)
  false_controlCattn = torch.clone(controlCattn)
  indices = torch.randperm(false_controlCattn.shape[-1])
  false_controlCattn = false_controlCattn[:,:,indices]

  dataset_controlCattn = TensorDataset(torch.cat((controlCattn, false_controlCattn),0),
                                      torch.cat((torch.ones(730,), torch.zeros(730,)),0))
  dataset_controlCattn = torch.utils.data.random_split(dataset_controlCattn, [0.9, 0.1])
  controlCattntrain = DataLoader(dataset_controlCattn[0], batch_size, shuffle=True)
  controlCattnval = DataLoader(dataset_controlCattn[1], batch_size, shuffle=True)

  fitschema(modelBschema, controlCattntrain, controlCattnval, tname+"BCs_controlattn", n_epochs=800, policy_only=True)
  fitcontrol(modelBcontrol, controlCattntrain, controlCattnval, tname+"BCc_controlattn", n_epochs=800)

  evaluate(modelBschema, controlCattnval, tname+"BCschema_controlattn", save_attn=False)
  evaluate(modelBcontrol, controlCattnval, tname+"BCcontrol_controlattn", save_attn=False)

  seedn += 1
  torch.manual_seed(seeds[seedn])

  del(controlCattn)
  del(false_controlCattn)
  del(indices)
  del(dataset_controlCattn)
  del(controlCattntrain)
  del(controlCattnval)
  del(modelBschema)
  del(modelBcontrol)
  del(modelBschema_wts)
  del(modelBcontrol_wts)
